# 1. Install Yolo

In [1]:
!pip install ultralytics

In [2]:
import torch

# 1. Cek apakah CUDA tersedia
if torch.cuda.is_available():
    print("✅ CUDA (GPU) tersedia!")
    
    # 2. Dapatkan jumlah GPU
    gpu_count = torch.cuda.device_count()
    print(f"    Jumlah GPU yang terdeteksi: {gpu_count}")
    
    # 3. Tampilkan informasi untuk setiap GPU
    for i in range(gpu_count):
        print(f"    --- GPU {i} ---")
        print(f"    Nama GPU: {torch.cuda.get_device_name(i)}")

    # 4. Tes komputasi sederhana di GPU pertama (GPU:0)
    try:
        print("\n🚀 Mencoba melakukan komputasi di GPU:0...")
        # Pindahkan tensor (data) ke GPU
        device = torch.device("cuda:0")
        tensor_cpu = torch.randn(3, 3)
        tensor_gpu = tensor_cpu.to(device)
        
        # Lakukan operasi di GPU
        result_gpu = tensor_gpu * tensor_gpu
        
        print("    Tensor asli (di CPU):\n", tensor_cpu)
        print("    Tensor setelah dipindah ke GPU:\n", tensor_gpu)
        print("    Hasil perkalian (di GPU):\n", result_gpu)
        print("    Komputasi di GPU berhasil!")
    except Exception as e:
        print(f"    ❌ Gagal melakukan komputasi di GPU: {e}")

else:
    print("❌ CUDA (GPU) tidak tersedia. Kode akan berjalan di CPU.")

✅ CUDA (GPU) tersedia!
    Jumlah GPU yang terdeteksi: 1
    --- GPU 0 ---
    Nama GPU: NVIDIA L4

🚀 Mencoba melakukan komputasi di GPU:0...
    Tensor asli (di CPU):
 tensor([[ 0.1898, -1.5446, -0.8666],
        [ 0.8330,  2.5887,  1.0925],
        [ 1.0960,  0.5953, -0.1644]])
    Tensor setelah dipindah ke GPU:
 tensor([[ 0.1898, -1.5446, -0.8666],
        [ 0.8330,  2.5887,  1.0925],
        [ 1.0960,  0.5953, -0.1644]], device='cuda:0')
    Hasil perkalian (di GPU):
 tensor([[0.0360, 2.3857, 0.7509],
        [0.6939, 6.7012, 1.1935],
        [1.2012, 0.3544, 0.0270]], device='cuda:0')
    Komputasi di GPU berhasil!


# 2. Import Library

In [3]:
import os
from ultralytics import YOLO

# 3. Data Preparation

In [4]:
# from google.colab import files
# uploaded = files.upload()

# Unzip file dataset
!unzip -o dataset.zip -d .

Archive:  dataset.zip
  inflating: ./README.dataset.txt    
  inflating: ./README.roboflow.txt   
  inflating: ./data.yaml             
 extracting: ./test/images/-4k-Relaxing-walk-in-Tokyo-Nakanosakaue-Shinjuku-1-_mp4-10_jpg.rf.f67bf1a011f7affa0b9882a97c24f67f.jpg  
 extracting: ./test/images/-4k-Relaxing-walk-in-Tokyo-Nakanosakaue-Shinjuku-1-_mp4-73_jpg.rf.f40f3b7d4a28fd54592c1f6f0d872c24.jpg  
 extracting: ./test/images/-Brainstorm-Video-Analysis-of-a-Runner_mp4-37_jpg.rf.4720dba337e8900d4de812a8978eb9f1.jpg  
 extracting: ./test/images/-Brainstorm-Video-Analysis-of-a-Runner_mp4-3_jpg.rf.a35eccecb54516743a4ac4e67260cd49.jpg  
 extracting: ./test/images/-Brainstorm-Video-Analysis-of-a-Runner_mp4-51_jpg.rf.fb6735cef7a3dc487fea1c70afa1f15b.jpg  
 extracting: ./test/images/-Brainstorm-Video-Analysis-of-a-Runner_mp4-55_jpg.rf.7e87bf67a66c75b0c48c638384e17a2d.jpg  
 extracting: ./test/images/-Brainstorm-Video-Analysis-of-a-Runner_mp4-6_jpg.rf.1d4ae70572d7dc9c9bcc23c5b1151b5d.jpg  
 extrac

In [5]:
print("Isi data/:", os.listdir("."))
print("Isi data/:", os.listdir("train"))
print("Isi data/:", os.listdir("valid"))
print("Isi data/:", os.listdir("test"))

Isi data/: ['train', 'README.roboflow.txt', 'valid', 'runs', 'dataset.zip', 'Copy_of_Begitulahipynb.ipynb', 'test', 'venv', '.ipynb_checkpoints', 'yolo12m.pt', 'README.dataset.txt', 'yolo11n.pt', 'data.yaml']
Isi data/: ['images', 'labels.cache', 'labels']
Isi data/: ['images', 'labels.cache', 'labels']
Isi data/: ['images', 'labels']


In [6]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


# 4. Data Training

In [7]:
import os
import urllib.request
from ultralytics import YOLO

# nama file weight
# weights_path = "yolo12m.pt"
weights_path = "yolo12m.pt"

# URL resmi ultralytics (ganti kalau ada link khusus YOLO12)
# weights_url = "https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo12m.pt"
# weights_url = "https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo12s.pt"

# # cek apakah file ada
# if not os.path.exists(weights_path):
#     print(f"{weights_path} tidak ditemukan, mengunduh dari {weights_url} ...")
#     urllib.request.urlretrieve(weights_url, weights_path)
#     print("✅ Download selesai.")
# else:
#     print("sudah ada")

# print("📦 Load model...")
model = YOLO(weights_path)
# model = YOLO(yolo12s.pt)

# data.yaml
data_yaml = "data.yaml"

# Train
model.train(
    data="data.yaml",          # dataset config
    device="cuda",             # GPU
    epochs=100,                # panjang biar converge
    imgsz=640,                 # image size
    batch=32,                  # lebih besar (L4 kuat, RAM 32GB aman)
    workers=8,                 # naikkan, lebih cepat loading data
    patience=25,               # early stopping
    save=True,                 # simpan best model
    project="runs/train",
    name="aic-train",
    exist_ok=True,
    cache=True,                # cache ke RAM
    amp=True,                  # mixed precision, hemat VRAM                         
    save_period=10,            # checkpoint tiap 10 epoch
)

print("✅ Training selesai")


Ultralytics 8.3.184 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA L4, 22574MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=aic-train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=25, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False

train: Scanning /home/g13222033/jupyterlab/train/labels.cache... 6760 images, 0 backgrounds, 0 corrupt: 100%|██████████

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 20, len(boxes) = 10307. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (7.7GB RAM): 100%|██████████| 6760/6760 [00:47<00:00, 143.23it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 729.6±353.9 MB/s, size: 27.1 KB)


...

    100/100      21.6G     0.3974     0.2176     0.8961         11        640: 100%|██████████| 212/212 [03:53<00:00,  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:10

                   all        785       1255      0.653      0.633       0.66      0.494



100 epochs completed in 6.814 hours.
Optimizer stripped from runs/train/aic-train/weights/last.pt, 40.8MB
Optimizer stripped from runs/train/aic-train/weights/best.pt, 40.8MB

Validating runs/train/aic-train/weights/best.pt...
Ultralytics 8.3.184 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA L4, 22574MiB)
YOLOv12m summary (fused): 169 layers, 20,120,332 parameters, 0 gradients, 67.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:10


                   all        785       1255      0.737      0.694      0.728      0.529
               Cycling          1          1          1          0      0.332      0.226
               Dancing          1          1          1          0      0.332      0.133
              Drinking         43         43      0.781      0.914      0.955      0.658
               Falling         90         94      0.956          1      0.995      0.829
               Jumping         33         33      0.577      0.879      0.783      0.591
               Picking         52         52      0.794      0.923       0.91      0.619
               Pouring         57         57      0.653      0.632      0.618       0.43
               Pushing         40         40      0.399       0.55      0.485       0.31
               Running        113        279       0.86      0.972      0.976      0.812
               Sitting         53         53      0.387       0.62        0.4      0.263
              Sleepin

In [8]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.get_device_name(0))


2.8.0+cu128
12.8
NVIDIA L4


# 5. Data Evaluation

In [14]:
import pandas as pd
import matplotlib.pyplot as plt

csv_path = "runs/train/aic-train/results.csv"
df = pd.read_csv(csv_path)

print(df.columns)

# Plot Box Loss (line chart masih masuk akal)
plt.plot(df["epoch"], df["train/box_loss"], label="Train Box Loss")
plt.plot(df["epoch"], df["val/box_loss"], label="Val Box Loss")
plt.xlabel("Epoch")
plt.ylabel("Box Loss")
plt.legend()
plt.title("Box Loss")
plt.show()

# Plot mAP (line chart biar keliatan tren per epoch)
plt.plot(df["epoch"], df["metrics/mAP50(B)"], label="mAP@0.5")
plt.plot(df["epoch"], df["metrics/mAP50-95(B)"], label="mAP@0.5:0.95")
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.legend()
plt.title("Validation mAP")
plt.show()

# Precision & Recall per epoch (line chart juga)
plt.plot(df["epoch"], df["metrics/precision(B)"], label="Precision")
plt.plot(df["epoch"], df["metrics/recall(B)"], label="Recall")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.legend()
plt.title("Precision & Recall")
plt.show()

# Kalau mau bar chart di epoch terakhir
last_epoch = df.iloc[-1]   # ambil data epoch terakhir
plt.bar(["Precision", "Recall"],
        [last_epoch["metrics/precision(B)"], last_epoch["metrics/recall(B)"]])
plt.ylabel("Score")
plt.title(f"Precision & Recall (Epoch {int(last_epoch['epoch'])})")
plt.show()


Index(['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss',
       'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)',
       'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss',
       'lr/pg0', 'lr/pg1', 'lr/pg2'],
      dtype='object')


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [11]:
from google.colab import files
files.download('runs/train/aic-train/weights/best.pt')

ModuleNotFoundError: No module named 'google'

In [12]:
# load model hasil training pertama (cek path dari output sebelumnya: save_dir, tambahin /weight/best.pt)
data_test = "test/images"
model_trained = YOLO("runs/train/aic-train/weights/best.pt")

results = model_trained.predict(
    source=data_test,
    conf=0.5,
    save=True,        # simpan gambar dengan bounding box
    save_txt=True,    # simpan label YOLO (.txt)
    project="aic-test",
    name="exp"
)



image 1/133 /home/g13222033/jupyterlab/test/images/-4k-Relaxing-walk-in-Tokyo-Nakanosakaue-Shinjuku-1-_mp4-10_jpg.rf.f67bf1a011f7affa0b9882a97c24f67f.jpg: 640x640 3 Walkings, 16.3ms
image 2/133 /home/g13222033/jupyterlab/test/images/-4k-Relaxing-walk-in-Tokyo-Nakanosakaue-Shinjuku-1-_mp4-73_jpg.rf.f40f3b7d4a28fd54592c1f6f0d872c24.jpg: 640x640 3 Standings, 1 Walking, 16.0ms
image 3/133 /home/g13222033/jupyterlab/test/images/-Brainstorm-Video-Analysis-of-a-Runner_mp4-37_jpg.rf.4720dba337e8900d4de812a8978eb9f1.jpg: 640x640 4 Runnings, 16.1ms
image 4/133 /home/g13222033/jupyterlab/test/images/-Brainstorm-Video-Analysis-of-a-Runner_mp4-3_jpg.rf.a35eccecb54516743a4ac4e67260cd49.jpg: 640x640 4 Runnings, 16.2ms
image 5/133 /home/g13222033/jupyterlab/test/images/-Brainstorm-Video-Analysis-of-a-Runner_mp4-51_jpg.rf.fb6735cef7a3dc487fea1c70afa1f15b.jpg: 640x640 4 Runnings, 15.6ms
image 6/133 /home/g13222033/jupyterlab/test/images/-Brainstorm-Video-Analysis-of-a-Runner_mp4-55_jpg.rf.7e87bf67a66c7

In [13]:
# Tampilkan prediksi
import matplotlib.pyplot as plt
import cv2
import glob


files = glob.glob("aic-test/exp/*.jpg") + glob.glob("aic-test/exp/*.png") # Cek posisi path dari output sebelumnya
print("Jumlah hasil prediksi:", len(files))

# tampilkan 9 gambar pertama
plt.figure(figsize=(15, 15))
for i, file in enumerate(files[:9]):
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.subplot(3, 3, i+1)
    plt.imshow(img)
    plt.axis("off")
    plt.title(os.path.basename(file))
plt.show()


Jumlah hasil prediksi: 133


<Figure size 1500x1500 with 9 Axes>